In [1]:
from pxr import Gf, Kind, Sdf, Usd, UsdGeom, UsdShade

In [7]:
import os
usd_scene_dir = '..\usd_scenes'
usd_billboard_filename='billboard.usda'
usd_billboard_filepath = os.path.join(usd_scene_dir, usd_billboard_filename)

# Create a component

## Kinds

* Primitives in a stage can grouped in kinds. This allows us to operate and filter by groups of primitives.
* This are the type of kinds
  * Assemblies: The root categories that are composed of multiples Assemblies or Components. For instance a campus
  * Component: Components are groups of Sub-Components and/or Meshes. Components atomic. You can't have a component that contains another component. For instance a tree in the campus
  * Sub-Component: A sub set of a component. For instance a leaf.


In [8]:
# Execute only if you have already created a stage and need to redo it.
try:
    del stage
except NameError:
    pass

In [9]:
stage = Usd.Stage.CreateNew(usd_billboard_filepath)
UsdGeom.SetStageUpAxis(stage, UsdGeom.Tokens.y)

billboard = UsdGeom.Xform.Define(stage, "/billboard")
Usd.ModelAPI(billboard).SetKind(Kind.Tokens.component)

True

##  Define a triangle mesh
* Vertices
* Faces
* Texture coordinates

In [10]:
billboard_mesh = UsdGeom.Mesh.Define(stage, "/billboard/mesh")
billboard_mesh.CreatePointsAttr([(-430, -145, 0), (430, -145, 0), (430, 145, 0), (-430, 145, 0)])
billboard_mesh.CreateFaceVertexCountsAttr([4])
billboard_mesh.CreateFaceVertexIndicesAttr([0,1,2,3])
billboard_mesh.CreateExtentAttr([(-430, -145, 0), (430, 145, 0)])

# Create a variable that varies across the primitive and needs to be interpolated
texCoords = billboard_mesh.CreatePrimvar("st", 
                                    Sdf.ValueTypeNames.TexCoord2fArray, 
                                    UsdGeom.Tokens.varying)
texCoords.Set([(0, 0), (1, 0), (1,1), (0, 1)])
stage.Save()
print stage.GetRootLayer().ExportToString()

#usda 1.0
(
    upAxis = "Y"
)

def Xform "billboard" (
    kind = "component"
)
{
    def Mesh "mesh"
    {
        float3[] extent = [(-430, -145, 0), (430, 145, 0)]
        int[] faceVertexCounts = [4]
        int[] faceVertexIndices = [0, 1, 2, 3]
        point3f[] points = [(-430, -145, 0), (430, -145, 0), (430, 145, 0), (-430, 145, 0)]
        texCoord2f[] primvars:st = [(0, 0), (1, 0), (1, 1), (0, 1)] (
            interpolation = "varying"
        )
    }
}




In [11]:
!usdview {usd_billboard_filepath}

# Make a material
* In USD, we "bind" geometry to Material prims in order to customize how the geometry should be shaded.  
* A Material is a container for networks of shading prims; a Material can contain one network that defines the surface illuminance response, and another that defines surface displacement.  
* It can also contain different networks for different renderers that don't share a common shading language. 
* Complex models will define multiple materials, binding different geometry (Gprims) to different Materials.

In [12]:
material = UsdShade.Material.Define(stage, '/mesh/boardmat')

## Create a surface

In [13]:
pbrShader = UsdShade.Shader.Define(stage, '/mesh/boardmatt/pbrshader')
pbrShader.CreateIdAttr("UsdPreviewSurface")
pbrShader.CreateInput("roughness", Sdf.ValueTypeNames.Float).Set(0.4)
pbrShader.CreateInput("metallic", Sdf.ValueTypeNames.Float).Set(0.0)

material.CreateSurfaceOutput().ConnectToSource(pbrShader, "surface")

True